In [2]:
ring1_state = ['GGGrrrrrrrrrrrrrrrrr','yyyrrrrrrrrrrrrrrrrr','rrrrrrrrrrrrrrrrrrrr', #0,1,2
                'rrrrrrrrrrrrrGGrrrrr','rrrrrrrrrrrrryyrrrrr','rrrrrrrrrrrrrrrrrrrr', #13,14
                'rrrrrrrrrrrrrrrGGGrr','rrrrrrrrrrrrrrryyyrr','rrrrrrrrrrrrrrrrrrrr', #15,16,17
                'rrrrrrrrGGrrrrrrrrrr','rrrrrrrryyrrrrrrrrrr','rrrrrrrrrrrrrrrrrrrr'] #8,9
ring2_state = ['rrrrrrrrrrGGGrrrrrrr','rrrrrrrrrryyyrrrrrrr','rrrrrrrrrrrrrrrrrrrr', #10,11,12
                'rrrGGrrrrrrrrrrrrrrr','rrryyrrrrrrrrrrrrrrr','rrrrrrrrrrrrrrrrrrrr', #3,4
                'rrrrrGGGrrrrrrrrrrrr','rrrrryyyrrrrrrrrrrrr','rrrrrrrrrrrrrrrrrrrr', #5,6,7
                'rrrrrrrrrrrrrrrrrrGG','rrrrrrrrrrrrrrrrrryy','rrrrrrrrrrrrrrrrrrrr'] #18,19

phaseAndGreenStatePairs = {"J1": [0,1,2], "J2": [13,14], "J3": [15,16,17], "J4": [8,9], "J5": [10,11,12], "J6": [3,4], "J7": [5,6,7], "J8": [18,19]}
phaseStrDict = {0: "J1", 1: "J2", 2: "J3", 3: "J4",4: "J5", 5: "J6", 6: "J7", 7: "J8"}
phaseStrDict_rev = {"J1": 0, "J2": 1, "J3": 2, "J4": 3, "J5": 4, "J6": 5, "J7": 6, "J8": 7}


In [40]:
class Phase:
    def __init__(self, phaseID, phaseOrder, startTime, green, yellow, allRed):
        self.phaseOrder = phaseOrder
        self.PhaseID = phaseID
        self.startTime = startTime
        self.green = green
        self.yellow = yellow
        self.allRed = allRed

    def setAllParameters(self,order,startTime,green,yellow,allRed):
        self.startTime = startTime
        self.green = green
        self.yellow = yellow
        self.allRed = allRed

    def setPhaseOrder(self,order):
        phaseOrderName = ["北往南直右", "南往北左轉", "西往東直右", "東往西左轉",
                          "南往北直右", "北往南左轉", "東往西直右", "西往東左轉", ]  # 8個分相
        self.phaseOrder = order
        self.name = phaseOrderName[order]

    def setStartTime(self,startTime):
        self.startTime = startTime
    # def setPhaseName(self,name):
    #     self.name = name
    def setPhaseGreen(self,green):
        self.green = green
    def setClearenceTime(self,yellow,allRed):
        self.yellow = yellow
        self.allRed = allRed

    def __str__(self):
        return 'PhaseObject(Order = {0}, name = {1}, startTime = {2}, green = {3}, yellow = {4}, allRed = {5})'\
            .format(self.phaseOrder, self.name, self.startTime, self.green, self.yellow, self.allRed)


In [41]:
Plan1 = [[34, 11, 43, 12, 34, 11, 43, 12], [0, 39, 55, 103, 0, 39, 55, 103]]
def create_phase(inputPlan):
    phase_list = []    
    for i in range(8):
        # using f-strings directly in the constructor call won't work because f-strings are evaluated at runtime, 
        # and the constructor call is executed during object initialization.
        # phaseID = f"J{i}"
        phase_list.append(Phase(phaseID=f"J{i}", phaseOrder=i, startTime=inputPlan[1][0], green=inputPlan[0][0], yellow=3, allRed=2))
    return phase_list


In [57]:
phases = create_phase(Plan1)
def splitDualRingtoTLS(phases):
    ring1 = []
    ring2 = []
    for phase in phases:
        if (phase.PhaseID in ["J1", "J2", "J3", "J4"]):
            try:
                phaseList = [phase.green, phase.yellow, phase.allRed]
                ring1.extend(phaseList)
            except AttributeError as error:
                print("ring 1. Attribute error")
        else:
            try:
                phaseList = [phase.green, phase.yellow, phase.allRed]
                ring2.extend(phaseList)
            except AttributeError as error:
                print("ring 2. Attribute error")
    combined_state = {"ring1":[], "ring2":[]}
    finalSignalPlan = {"state":[], "duration":[]}
    # print(f"ring1 phase duration:{ring1}")

    for index in range(1, len(ring2)):
        ring1[index] = ring1[index - 1] + ring1[index]
        ring2[index] = ring2[index - 1] + ring2[index]
    # print(f"ring1 phase start time:{ring1}")
    # it will create a new list that contains all the elements from both lists in the order they appear.
    ringSplitTime = ring1 + ring2
    print(f"splitTime:{ringSplitTime}")
    ringSplitTime.sort()
    ringSplitTime = list(dict.fromkeys(ringSplitTime))

    def splitRingTime(ring):
        ringIndex = 0
        if ring == "ring1":
            for item in ringSplitTime:
                print(f"ring1 combined_state:{combined_state}")
                if (item < ring1[ringIndex]):
                    combined_state[ring].append(ring1_state[ringIndex])
                elif (item == ring1[ringIndex]):
                    combined_state[ring].append(ring1_state[ringIndex])
                    ringIndex = ringIndex + 1
                else:
                    ringIndex = ringIndex + 1
                    combined_state[ring].append(ring1_state[ringIndex])
        else:
            for item in ringSplitTime:
                if (item < ring2[ringIndex]):
                    combined_state[ring].append(ring2_state[ringIndex])
                elif (item == ring2[ringIndex]):
                        combined_state[ring].append(ring2_state[ringIndex])
                        ringIndex = ringIndex + 1
                else:
                    ringIndex = ringIndex + 1
                            # print("ring2_state = ", ring2_state)
                            # print("len  = ",len(ring2_state))
                            # print("ringIndex = ", ringIndex)
                    try:
                        combined_state[ring].append(ring2_state[ringIndex])
                    except IndexError as e:
                        print("抓!PhaseObject 有IndexError例外錯誤! 記得內審完後來修我!")
                        print(e)
                    return True
                
    ringTime1 = splitRingTime("ring1")
    
    ringTime2 = splitRingTime("ring2")

    return ringTime1, ringTime2


In [58]:
x,y = splitDualRingtoTLS(phases)

splitTime:[34, 37, 39, 73, 76, 78, 112, 115, 117, 151, 154, 156, 34, 37, 39, 73, 76, 78, 112, 115, 117, 151, 154, 156]
ring1 combined_state:{'ring1': [], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr', 'yyyrrrrrrrrrrrrrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr', 'yyyrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrrrrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr', 'yyyrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrGGrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr', 'yyyrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrGGrrrrr', 'rrrrrrrrrrrrryyrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr', 'yyyrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrrrrrrrr', 'rrrrrrrrrrrrrGGrrrrr', 'rrrrrrrrrrrrryyrrrrr', 'rrrrrrrrrrrrrrrrrrrr'], 'ring2': []}
ring1 combined_state:{'ring1': ['GGGrrrrrrrrrrrrrrrrr

In [ ]:
def splitRingTime(ring_name, ringSplitTime):
    ringIndex = 0
    if ring_name == 'ring1':
        for item in ringSplitTime:
            if (item < ring1[ringIndex]):
                combined_state[ring_name]
    